In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from imageio import imread
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
train_dir = "C:/Users/dedsec995/Projects/tensorflow/breed/data/train"
valid_dir = "C:/Users/dedsec995/Projects/tensorflow/breed/data/test/"
print(os.listdir("C:/Users/dedsec995/Projects/tensorflow/breed/data"))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/dedsec995/Projects/tensorflow/breed/data'

In [10]:
# from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from collections import Counter
from scipy.ndimage import *
from scipy.io import loadmat
from scipy.misc import *
import os
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import numpy

In [11]:
base_model = Xception(weights='imagenet', include_top=False)

In [12]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dropout(0.5)(x)
x = Dense(100, activation='relu')(x)
# This layer being at 1024 isn't bad. ^
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(37, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [13]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator(
                                   rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode='nearest'
)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(
                                    rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode='nearest'
)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size  = 8,
                                                    target_size = (224, 224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( valid_dir,
                                                          batch_size  = 8, 
                                                          target_size = (224, 224))

Found 6682 images belonging to 37 classes.
Found 707 images belonging to 37 classes.


In [14]:
history = model.fit(train_generator,
#                               steps_per_epoch=2000,
                              validation_data=validation_generator,
                              epochs=10,
#                               validation_steps=5,
                              verbose=1)

Epoch 1/10
836/836 [==============================] - 91s 109ms/step - loss: 1.4229 - accuracy: 0.6078 - val_loss: 0.9329 - val_accuracy: 0.7143
Epoch 2/10
836/836 [==============================] - 80s 96ms/step - loss: 0.8429 - accuracy: 0.7438 - val_loss: 0.7949 - val_accuracy: 0.7412
Epoch 3/10
836/836 [==============================] - 82s 98ms/step - loss: 0.7726 - accuracy: 0.7634 - val_loss: 0.7872 - val_accuracy: 0.7496
Epoch 4/10
836/836 [==============================] - 91s 108ms/step - loss: 0.7170 - accuracy: 0.7766 - val_loss: 0.7667 - val_accuracy: 0.7610
Epoch 5/10
836/836 [==============================] - 86s 103ms/step - loss: 0.6925 - accuracy: 0.7814 - val_loss: 0.7954 - val_accuracy: 0.7440
Epoch 6/10
836/836 [==============================] - 85s 102ms/step - loss: 0.6604 - accuracy: 0.7869 - val_loss: 0.7724 - val_accuracy: 0.7652
Epoch 7/10
836/836 [==============================] - 85s 101ms/step - loss: 0.6400 - accuracy: 0.7951 - val_loss: 0.7783 - val_accu